In [56]:
import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos gráficos de probabilidad
import pgmpy.factors.discrete as pgmf  # Tablas de probabilidades condicionales y factores de probabilidad
import pgmpy.inference as pgmi  # Inferencia probabilística exacta
import random
import numpy 

In [57]:
X = 5
Y = 5
numero_minas = 5

In [58]:
#RESPECTO AL TABLERO
def crear_tablero (X, Y, numero_minas):
    #Si hay mas minas que posiciones del tablero salta un error
    if (numero_minas > X * Y):
        raise Exception('No puede haber más minas que casillas!!')
        #esto saltaria si tuvieramos un boton,o alguna forma de preguntarlo,si pusieramos de mas ?
        
    #Ahora construyo el tablero con el tamaño definido por 'X' e 'Y'.
    tablero = [[0] * X for i in range(X)]
    
    #Colocamos las minas en el tablero de forma aleatoria
    for i in range (numero_minas):
        columna_aleatoria = -1
        fila_aleatoria = -1
        #Si no tiene asignada ninguna columna ni fila se le pone una mina en una posicion aleatoria
        while (columna_aleatoria == -1 or fila_aleatoria == -1 or tablero[fila_aleatoria][columna_aleatoria] == "M"):
            columna_aleatoria = random.randint (0, Y-1)
            fila_aleatoria = random.randint (0, X-1)
            
            #Se van colocando las minas    
        tablero[fila_aleatoria][columna_aleatoria] = "M" #Se le asigna una mina con la letra M
    #Contamos el numero de minas vecinas para cada casilla 
    #Contamos el numero de minas vecinas para cada casilla para completar el tablero
    #Dibujo explicativo de los índices vecinos: [posicion del condicional](i,j)
    #  |[1](-1,-1)|[2](-1,0)|[3](-1,+1)|
    #  |[4](0,-1) |[-]( 0,0)|[5] (0,+1)| //CASILLA CENTRAL ELEGIDA 
    #  |[6](+1,-1)|[7](+1,0)|[8](+1,+1)|
    suma = 0
    for i in range (X): 
        for j in range (Y):
            if ((i-1) >= 0 and (j-1) >= 0 and tablero[i-1][j-1] == "M"): #1
                suma = suma + 1
            if ((i-1) >= 0 and tablero[i-1][j] == "M"): #2
                suma = suma + 1
            if ((i-1) >= 0 and (j+1) < len(tablero) and tablero[i-1][j+1]  == "M"): #3 
                suma = suma + 1
            if ((j-1) >= 0 and tablero[i][j-1] == "M"): #4
                suma = suma + 1
            if ((j+1) < len(tablero) and tablero[i][j+1] == "M"): #5
                suma = suma + 1
            if ((i+1) < len(tablero) and (j-1) >= 0 and tablero[i+1][j-1] == "M"): #6
                suma = suma + 1
            if ((i+1) < len(tablero) and tablero[i+1][j] == "M"): #7
                suma = suma + 1
            if ((i+1) < len(tablero) and (j+1) < len(tablero) and tablero[i+1][j+1] == "M"): #8
                suma = suma + 1
            #Si la suma de minas vecinas es mayor que 0 y esa casilla no es una mina: se guarda ese numero en esa posicion
            if (suma > 0 and tablero[i][j] != 'M'):
                tablero[i][j] = str(suma)
            suma = 0 
            
    return tablero
    #Quedaría hacer las esquinas y lados de casilla escogida!!

#OCULTAMOS TODAS LAS CASILLAS PARA INICIALIZAR EL TABLERO NO VISIBLE
def tablero_oculto (X):
    tableroOculto = [['X'] * X for i in range(X)]   # He definido que las casillas ocultas van a estar con una X
    return tableroOculto

#RESPECTO A LAS CASILLAS
def destapar_casilla (i, j, tableroOculto, tableroDestapado):
    if (tableroOculto[i][j] == 'X'):
        casillaResuelta = tableroDestapado[i][j] #guardo en esa variable el resultado de la casilla destapada del tablero
        #Ahora vemos que sucede en funcion de lo que de al destapar esa casilla
        #Si la casilla es una mina
        #  |[1](-1,-1)|[2](-1,0)|[3](-1,+1)|
        #  |[4](0,-1) |[-]( 0,0)|[5] (0,+1)|
        #  |[6](+1,-1)|[7](+1,0)|[8](+1,+1)|
        if(casillaResuelta == 'M'):
            raise Exception('Has perdido! ha explotado una mina')
        #Si la casilla No es una mina
        else:
            tableroOculto[i][j] = casillaResuelta
        #Si la casilla NO es una mina y tiene un 0 se destapa la zona
        if(casillaResuelta == 0): #ahora de la misma manera que los if anteriores se destapan dependiendo de la posicion relativa de la casilla 
            if((i-1)>=0 and (j-1)>=0):#1
                destapar_casilla(i-1, j-1, tableroOculto, tableroDestapado)
            if((i-1)>=0):#2
                destapar_casilla(i-1, j, tableroOculto, tableroDestapado) 
            if((i-1)>=0 and (j+1) < len(tableroOculto)):#3
                destapar_casilla(i-1, j+1, tableroOculto, tableroDestapado)
            if((j-1)>=0):#4
                destapar_casilla(i, j-1, tableroOculto, tableroDestapado)
            if((j+1)<len(tableroOculto)):#5
                destapar_casilla(i, j+1, tableroOculto, tableroDestapado)
            if((i+1)<len(tableroOculto) and (j-1)>=0):#6
                destapar_casilla(i+1, j-1, tableroOculto, tableroDestapado)
            if((i+1)<len(tableroOculto)):#7
                destapar_casilla(i+1, j, tableroOculto, tableroDestapado)
            if((i+1)<len(tableroOculto) and (j+1)<len(tableroOculto)):#8
                destapar_casilla(i+1, j+1, tableroOculto, tableroDestapado)
                     
        return tableroOculto


In [59]:
tableroDestapado = crear_tablero (X, Y, numero_minas)

In [60]:
tableroOculto = tablero_oculto(X)

In [61]:
print(numpy.array(tableroDestapado))

[['1' '1' '1' '0' '0']
 ['1' 'M' '1' '1' '1']
 ['1' '1' '1' '2' 'M']
 ['1' '1' '1' '3' 'M']
 ['1' 'M' '1' '2' 'M']]


In [77]:
i = 1
j = 0

In [78]:
destaparCasilla = destapar_casilla(i, j, tableroOculto, tableroDestapado)

In [79]:
print(numpy.array(destaparCasilla)) #Se realiza la accion de destapar una casilla del tablero oculto

[['1' 'X' 'X' 'X' 'X']
 ['1' 'X' 'X' 'X' 'X']
 ['X' 'X' 'X' 'X' 'X']
 ['X' 'X' 'X' 'X' 'X']
 ['X' 'X' 'X' 'X' 'X']]


In [80]:
def indices_casillas_vecinas (i, j, tableroDestapado): # De esta manera se añaden los índices de las casillas vecinas a la destapada.
    indiceCasillasVecinas = []
    
    if((i-1)>=0 and (j-1)>=0):#1
        indiceCasillasVecinas.append('{0}{1}'.format(i-1, j-1)) #base 10
    if((i-1)>=0):#2
        indiceCasillasVecinas.append('{0}{1}'.format(i-1, j)) 
    if((i-1)>=0 and (j+1) < len(tableroOculto)):#3
        indiceCasillasVecinas.append('{0}{1}'.format(i-1, j+1))
    if((j-1)>=0):#4
        indiceCasillasVecinas.append('{0}{1}'.format(i, j-1))
    if((j+1)<len(tableroOculto)):#5
        indiceCasillasVecinas.append('{0}{1}'.format(i, j+1))
    if((i+1)<len(tableroOculto) and (j-1)>=0):#6
        indiceCasillasVecinas.append('{0}{1}'.format(i+1, j-1))
    if((i+1)<len(tableroOculto)):#7
        indiceCasillasVecinas.append('{0}{1}'.format(i+1, j))
    if((i+1)<len(tableroOculto) and (j+1)<len(tableroOculto)):#8
        indiceCasillasVecinas.append('{0}{1}'.format(i+1, j+1))
    
    return indiceCasillasVecinas

In [81]:
print(indices_casillas_vecinas (i, j, tableroDestapado)) #no hace falta que sea sobre el tablero destapado (puede ser sobre el oculto). SOlo queremos los índices.

['00', '01', '11', '20', '21']


In [82]:
def lista_de_minas (tableroDestapado):
    ListaMinas = []
    for i in range (X):
        for j in range (Y):
            if(tableroDestapado[i][j] == 'M'):
                ListaMinas.append('{0}{1}'.format(i, j))
    return ListaMinas

In [83]:
print (lista_de_minas (tableroDestapado))

['11', '24', '34', '41', '44']


In [84]:
def crear_red_bayesiana(X, Y, tableroOculto):
    modelo_buscaminas = pgmm.BayesianModel()
    #x->oculta
    #Y->destapada
    ListaCompletaCasillasOcultas = []
    ListaCasillasOcultas = []
    ListaCasillasDestapadas = []
    for i in range (X):
        for j in range (Y):
            casillasVecinas = indices_casillas_vecinas(i, j, tableroDestapado) #En principio vamos a probar con destapado(aunque deberia dar igual porque son indices)
            if (tableroOculto[i][j] == 'X'):
                
                ListaCompletaCasillasOcultas.append('Oculta({0},{1})'.format(i, j)) 
                
                tieneVecinosDestapados = False
                for vecinoCasillaActual in casillasVecinas:
                    if(tableroOculto[int(vecinoCasillaActual[0])][int(vecinoCasillaActual[1])] != 'X'):
                        tieneVecinosDestapados = True
                        break
                if(tieneVecinosDestapados):
                    ListaCasillasOcultas.append('Oculta({0},{1})'.format(i, j))
                    modelo_buscaminas.add_nodes_from(['Oculta({0},{1})'.format(i, j)])    
            else:
                ListaCasillasDestapadas.append('Destapada({0},{1})'.format(i, j))
                
                tieneVecinosOcultos = False
                ListaDeMinas = lista_de_minas(tableroDestapado)
                for vecinoCasillaActual in casillasVecinas:
                    if(tableroOculto[int(vecinoCasillaActual[0])][int(vecinoCasillaActual[1])] != 'X' and not ListaDeMinas.__contains__(vecinoCasillaActual)):
                        tieneVecinosDestapados = True
                        break
                if(tieneVecinosOcultos):
                    ListaCasillasDestapadas.append('Destapada({0},{1})'.format(i, j))
                    modelo_buscaminas.add_nodes_from(['Destapada({0},{1})'.format(i, j)])
    
    print("Lista Completa casillas ocultas : ")
    print(ListaCompletaCasillasOcultas) 
    print("\n")
    print("Lista casillas vecinas ocultas acumuladas hasta la posicion actual : ")
    print(ListaCasillasOcultas)
    print("\n")
    print("Lista casillas destapadas hasta ese momento : ")
    print(ListaCasillasDestapadas)
    print("\n")
    
    print("Comprobamos la lista de nodos añadidas a la red bayesiana:")
    print(modelo_buscaminas.nodes())
    print("\n")
    #Una vez tenemos los nodos añadidos a la red bayesiana procedemos a añadir aristas entre los nodos.
    #De manera que las casillas destapadas tendran aristas con sus ocultas (vecinas)
    
    Aristas=dict() #dict ???
    for i in range(X):
        for j in range(Y):
            Destapada='Destapada({0},{1})'.format(i, j)
            #Si la casilla ha sido destapada y se ha añadido a la lista de casillas destapadas
            if(tableroOculto[i][j]!='X' and ListaCasillasDestapadas.__contains__(Destapada)):
                ListaVecinosOcultos=[]
                casillasVecinas=indices_casillas_vecinas (i, j, tableroDestapado) 
                #me recorro las vecinas de la destapada
                for vecinoCasillaActual in casillasVecinas:
                    Oculta='Oculta({0},{1})'.format(vecinoCasillaActual[0],vecinoCasillaActual[1])
                    #si es oculta la casilla vecina que me estoy recorriendo la añado al grafo
                    if(ListaCasillasOcultas.__contains__(Oculta)):
                        modelo_buscaminas.add_edges_from([(Oculta, Destapada)])
                        ListaVecinosOcultos.append(Oculta)
                Aristas.update({Destapada: ListaVecinosOcultos})#asocia la casilla destapada con la lista de vecinos ocultos
                
    print("Lista de aristas:")
    print(Aristas)
    print("\n")
    print("Lista de vecinos ocultos:")
    print(ListaVecinosOcultos)
    print("\n")
    
    print("Comprobamos la lista de aristas añadidas a la red bayesiana:") #Formato: [Oculta(i,j), Destapada(fijo,fijo)]
    print(modelo_buscaminas.edges())
    print("\n")
    
    return modelo_buscaminas

In [85]:
print(crear_red_bayesiana(X, Y, tableroOculto))


Lista Completa casillas ocultas : 
['Oculta(0,1)', 'Oculta(0,2)', 'Oculta(0,3)', 'Oculta(0,4)', 'Oculta(1,1)', 'Oculta(1,2)', 'Oculta(1,3)', 'Oculta(1,4)', 'Oculta(2,0)', 'Oculta(2,1)', 'Oculta(2,2)', 'Oculta(2,3)', 'Oculta(2,4)', 'Oculta(3,0)', 'Oculta(3,1)', 'Oculta(3,2)', 'Oculta(3,3)', 'Oculta(3,4)', 'Oculta(4,0)', 'Oculta(4,1)', 'Oculta(4,2)', 'Oculta(4,3)', 'Oculta(4,4)']


Lista casillas vecinas ocultas acumuladas hasta la posicion actual : 
['Oculta(0,1)', 'Oculta(1,1)', 'Oculta(2,0)', 'Oculta(2,1)']


Lista casillas destapadas hasta ese momento : 
['Destapada(0,0)', 'Destapada(1,0)']


Comprobamos la lista de nodos añadidas a la red bayesiana:
['Oculta(0,1)', 'Oculta(1,1)', 'Oculta(2,0)', 'Oculta(2,1)']


Lista de aristas:
{'Destapada(0,0)': ['Oculta(0,1)', 'Oculta(1,1)'], 'Destapada(1,0)': ['Oculta(0,1)', 'Oculta(1,1)', 'Oculta(2,0)', 'Oculta(2,1)']}


Lista de vecinos ocultos:
['Oculta(0,1)', 'Oculta(1,1)', 'Oculta(2,0)', 'Oculta(2,1)']


Comprobamos la lista de aristas añad

In [2]:

#Creo tabla de probabilidad(CPD)
#Las tablas de las ocultas no dependen de nada
#Las tablas de las destapadas dependen de las ocultas vecinas.
def crear_CPD(modelo_buscaminas, ListaMinas):
    
    #CPD de Ocultas
    ListaValoresDestapados=dict() 
    for i in range(X):
        for j in range(Y):
            Oculta = 'Oculta{0},{1}'.format(i,j)
            if(tableroOculto[i][j]=='X' and ListaCasillasOcultas.__contains__(Oculta)):
                #Si la casilla por destapar tiene la misma posicion que una de las minas
                if(ListaMinas.__contains__(Oculta)):
                    probabilidadMina = 1
                #Si el numero de casillas por destapar es mayor que le numero de minas que tiene el tablero
                elif(len(ListaCompletaCasillasOcultas)>numero_minas):
                    #numero de casillas destapadas = (X*Y)-len(ListaCompletaCasillasOcultas)
                    #La probabilidad de que sea mina es el (numero de casillas destapadas-numero de minas)/numero de casillas ocultas
                    probabilidadMina = ((X*Y)-len(ListaCompletaCasillasOcultas)-numero_minas/len(ListaCompletaCasillasOcultas))
                else:
                    probabilidadMina = 0
                    
                #Añadimos la cpd al modelo 
                cpd = pgmf.TabularCPD(Oculta, 2, [[1-probabilidadMina, probabilidadMina]]) #Esto hay que ponerlo bien
                modelo_buscaminas.add_cpds(cpd)                
                                    
                                
   